<a href="https://colab.research.google.com/github/andrewcgaitskell/colab-threejs-charting/blob/main/quart_threejs_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quart + Jinja2 + Three.js Chart Demo

This notebook demonstrates how to:
- Use **Pandas** to create sample data
- Build an async web server with **Quart**
- Render HTML templates with **Jinja2**
- Visualize data with **Three.js** 3D charts

**Goal**: Show a simple, clean pattern for building interactive 3D visualizations.


## 1. Install Dependencies

Install Quart (async web framework) and required packages.


In [1]:
!pip install -q quart pandas numpy requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.5 MB/s eta 0:00:00


## 2. Create Sample Data

Generate simple sales data using Pandas.

In [2]:
import pandas as pd
import numpy as np

# Create sample quarterly sales data for different products
data = {
    'Product': ['Product A', 'Product B', 'Product C', 'Product D'],
    'Q1': [45, 60, 38, 52],
    'Q2': [58, 72, 45, 61],
    'Q3': [62, 68, 51, 58],
    'Q4': [71, 85, 59, 67]
}

df = pd.DataFrame(data)
print("Sample Data:")
print(df)

# Convert to format suitable for JavaScript
chart_data = df.to_dict('records')
print("\nData for chart:", chart_data)

Sample Data:
     Product  Q1  Q2  Q3  Q4
0  Product A  45  58  62  71
1  Product B  60  72  68  85
2  Product C  38  45  51  59
3  Product D  52  61  58  67

Data for chart: [{'Product': 'Product A', 'Q1': 45, 'Q2': 58, 'Q3': 62, 'Q4': 71}, {'Product': 'Product B', 'Q1': 60, 'Q2': 72, 'Q3': 68, 'Q4': 85}, {'Product': 'Product C', 'Q1': 38, 'Q2': 45, 'Q3': 51, 'Q4': 59}, {'Product': 'Product D', 'Q1': 52, 'Q2': 61, 'Q3': 58, 'Q4': 67}]


## 3. Set Up Quart Application

Create a Quart app with a single route that renders our chart.

In [3]:
from quart import Quart, render_template_string
import json
import requests

app = Quart(__name__)

# Load HTML template from repository
TEMPLATE_URL = 'https://raw.githubusercontent.com/andrewcgaitskell/colab-threejs-charting/main/templates/chart_template.html'
response = requests.get(TEMPLATE_URL, timeout=10)
HTML_TEMPLATE = response.text

@app.route('/')
async def index():
    """Main route that renders the 3D chart with data"""
    return await render_template_string(HTML_TEMPLATE, data=chart_data)

print("✓ Quart app configured")


✓ Quart app configured


## 4. Run the Server

Start the Quart server. In Colab, the server will run on localhost.


In [6]:
import asyncio
from google.colab import output

# Enable port forwarding and display in iframe
output.serve_kernel_port_as_iframe(5000, height=600)

# Run the Quart app
await app.run_task(host='0.0.0.0', port=5000)


<IPython.core.display.Javascript object>

[2026-02-14 11:02:33 +0000] [384] [INFO] Running on http://0.0.0.0:5000 (CTRL + C to quit)
INFO:hypercorn.error:Running on http://0.0.0.0:5000 (CTRL + C to quit)


---

## Summary

This notebook demonstrates:

1. **Data Pipeline**: Pandas → Python dict → JSON → JavaScript
2. **Web Framework**: Quart (async) with Jinja2 templating
3. **3D Visualization**: Three.js for interactive charts

### Key Features:
- ✅ Simple, minimal code
- ✅ Interactive 3D controls (drag to rotate, scroll to zoom)
- ✅ Clean data flow from Python to JavaScript
- ✅ Professional presentation-ready output

### Customization Ideas:
- Change the data in Step 2
- Modify colors in the Three.js code
- Add more chart types (scatter, line, etc.)
- Enhance with additional Three.js features (labels, animations, etc.)
